In [1]:
import os
from pathlib import Path

import jieba
import numpy as np
import pandas as pd
from gensim.corpora import Dictionary
from gensim.models import TfidfModel, LdaModel
from tqdm import tqdm



In [8]:
# 获取该文件的绝对路径
base_path = os.getcwd()
print (base_path)
# 构建指向特定文件的路径
file_path = os.path.join(base_path,'datas/online_shopping_10_cats.csv')
print (file_path)

C:\Users\五行缺钱\PycharmProjects\pythonProject1\NLP\codes_005
C:\Users\五行缺钱\PycharmProjects\pythonProject1\NLP\codes_005\datas/online_shopping_10_cats.csv


In [4]:
# 创建docs两个文档每个文档中含有三个词汇
docs = [
    ["w1", "w2", "w1"],
    ["w1", "w2", "w3"]
]
# 创建一个词汇表 dct
dct = Dictionary(docs)
# 输出词汇表中不同词汇的数量
print(len(dct))
# 使用 dct.doc2bow() 方法将每个文档转换为词袋模型（Bag of Words）表示
# 词袋法即用矩阵的形式把词汇出现的次数和对应的索引表示出来
corpus = [dct.doc2bow(line) for line in docs]
print(corpus)

3
[[(0, 2), (1, 1)], [(0, 1), (1, 1), (2, 1)]]


In [10]:
def split_words(doc):
    # jieba分词
    # doc.strip("\ufeff")：这个表达式用于去除字符串开头的字节顺序标记（Byte Order Mark, BOM）。BOM 是一种特殊的字符，用于标识文件中使用的字符编码
    # 特别是在 UTF-8 编码的文件中。strip() 函数在这里用于去除字符串两端的空白字符和 BOM。
    words = jieba.lcut(doc.strip("\ufeff").strip())
    return words

In [11]:
# 将稀疏的词袋模型表示转换为一个密集的 NumPy 数组
def transformedCorpus2Numpy(corpus, n_docs, n_features):
    final_ret = np.zeros(shape=(n_docs, n_features))
    for i, ret in enumerate(corpus):
        # print(ret)
        for j, value in ret:
            final_ret[i][j] = value
    return final_ret

## Dictionary

In [21]:
data_file_path = file_path
# 定义输出目录的路径，该目录将用于保存词汇表和其他输出文件。
output_dir = os.path.join(base_path, "output/gensim/t1")
# 创建输出目录。如果目录已存在，则 exist_ok=True 参数确保不会抛出异常
os.makedirs(output_dir, exist_ok=True)
# 1. 词典构建
docs = []
df = pd.read_csv(data_file_path)

# 看看有没有提取到以及其对应的属性
print(df.head())
print(df['cat'].value_counts())
print(df['review'][0])
print(dir(docs))

# 从 DataFrame 中提取 review 列的值，这些值代表要处理的文档
documents = df.review.values
# 遍历文档列表，并使用 tqdm 显示进度条
for document in tqdm(documents):
    # document = str(document)  # 类型强制转换为str
    try:
        # 对每个文档进行分词，并将分词结果添加到 docs 列表中
        docs.append(split_words(document))
    except Exception as e:
        print(document, "异常:", e)
dct = Dictionary(docs)
# 将构建的词汇表保存为文本文件，保存在之前定义的输出目录中
dct.save_as_text(os.path.join(output_dir , "dict.txt"))

print(docs[0])
print(dct.token2id)

  cat  label                                             review
0  书籍      1  ﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持...
1  书籍      1  作者真有英国人严谨的风格，提出观点、进行论述论证，尽管本人对物理学了解不深，但是仍然能感受到...
2  书籍      1  作者长篇大论借用详细报告数据处理工作和计算结果支持其新观点。为什么荷兰曾经县有欧洲最高的生产...
3  书籍      1  作者在战几时之前用了＂拥抱＂令人叫绝．日本如果没有战败，就有会有美军的占领，没胡官僚主义的延...
4  书籍      1  作者在少年时即喜阅读，能看出他精读了无数经典，因而他有一个庞大的内心世界。他的作品最难能可贵...
cat
平板     10000
水果     10000
洗发水    10000
衣服     10000
酒店     10000
计算机     3992
书籍      3851
手机      2323
蒙牛      2033
热水器      575
Name: count, dtype: int64
﻿做父母一定要有刘墉这样的心态，不断地学习，不断地进步，不断地给自己补充新鲜血液，让自己保持一颗年轻的心。我想，这是他能很好的和孩子沟通的一个重要因素。读刘墉的文章，总能让我看到一个快乐的平易近人的父亲，他始终站在和孩子同样的高度，给孩子创造着一个充满爱和自由的生活环境。很喜欢刘墉在字里行间流露出的做父母的那种小狡黠，让人总是忍俊不禁，父母和子女之间有时候也是一种战斗，武力争斗过于低级了，智力较量才更有趣味。所以，做父母的得加把劲了，老思想老观念注定会一败涂地，生命不息，学习不止。家庭教育，真的是乐在其中。
['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', 

 61%|██████    | 38104/62774 [00:04<00:02, 12181.33it/s]

nan 异常: 'float' object has no attribute 'strip'


100%|██████████| 62774/62774 [00:07<00:00, 8416.88it/s] 


['做', '父母', '一定', '要', '有', '刘墉', '这样', '的', '心态', '，', '不断', '地', '学习', '，', '不断', '地', '进步', '，', '不断', '地', '给', '自己', '补充', '新鲜血液', '，', '让', '自己', '保持', '一颗', '年轻', '的', '心', '。', '我', '想', '，', '这', '是', '他', '能', '很', '好', '的', '和', '孩子', '沟通', '的', '一个', '重要', '因素', '。', '读', '刘墉', '的', '文章', '，', '总能', '让', '我', '看到', '一个', '快乐', '的', '平易近人', '的', '父亲', '，', '他', '始终', '站', '在', '和', '孩子', '同样', '的', '高度', '，', '给', '孩子', '创造', '着', '一个', '充满', '爱', '和', '自由', '的', '生活', '环境', '。', '很', '喜欢', '刘墉', '在', '字里行间', '流露出', '的', '做', '父母', '的', '那种', '小', '狡黠', '，', '让', '人', '总是', '忍俊不禁', '，', '父母', '和', '子女', '之间', '有时候', '也', '是', '一种', '战斗', '，', '武力', '争斗', '过于', '低级', '了', '，', '智力', '较量', '才', '更', '有', '趣味', '。', '所以', '，', '做', '父母', '的', '得', '加把劲', '了', '，', '老', '思想', '老', '观念', '注定', '会', '一败涂地', '，', '生命不息', '，', '学习', '不止', '。', '家庭教育', '，', '真的', '是', '乐在其中', '。']
{'。': 0, '一个': 1, '一定': 2, '一种': 3, '一败涂地': 4, '一颗': 5, '不断': 6, '不止': 7, '之间': 8, '乐在其中': 9, '也': 10, '

## doc2bow

In [22]:
# 2. 词袋法构建
corpus = [dct.doc2bow(line) for line in docs]
print(len(corpus))
print(corpus[0])

62773
[(0, 6), (1, 3), (2, 1), (3, 1), (4, 1), (5, 1), (6, 3), (7, 1), (8, 1), (9, 1), (10, 1), (11, 2), (12, 1), (13, 1), (14, 2), (15, 1), (16, 1), (17, 1), (18, 3), (19, 1), (20, 3), (21, 1), (22, 1), (23, 1), (24, 4), (25, 1), (26, 1), (27, 2), (28, 3), (29, 1), (30, 1), (31, 1), (32, 1), (33, 2), (34, 3), (35, 1), (36, 1), (37, 1), (38, 1), (39, 2), (40, 1), (41, 1), (42, 1), (43, 1), (44, 1), (45, 1), (46, 1), (47, 1), (48, 1), (49, 2), (50, 1), (51, 1), (52, 1), (53, 1), (54, 1), (55, 3), (56, 1), (57, 1), (58, 2), (59, 1), (60, 1), (61, 1), (62, 1), (63, 1), (64, 1), (65, 1), (66, 4), (67, 1), (68, 1), (69, 1), (70, 1), (71, 12), (72, 1), (73, 1), (74, 1), (75, 1), (76, 2), (77, 2), (78, 1), (79, 2), (80, 1), (81, 1), (82, 1), (83, 1), (84, 3), (85, 1), (86, 1), (87, 1), (88, 1), (89, 1), (90, 1), (91, 1), (92, 1), (93, 1), (94, 1), (95, 17)]


## TfidfModel

In [23]:
# 3. TFIDF模型构建
model = TfidfModel(corpus=corpus)
model.save(os.path.join(output_dir , "tfidf.pkl"))

In [25]:
output_dir = os.path.join(base_path, "output/gensim/t1")
# 创建输出目录，如果目录已存在，则不会抛出异常
os.makedirs(output_dir, exist_ok=True)

# 1. 加载字典
dct = Dictionary.load_from_text(os.path.join(output_dir , "dict.txt"))

# 2. 模型恢复
model = TfidfModel.load(os.path.join(output_dir , "tfidf.pkl"))

# 3. 当前样本预测
reviews = [
    "作为有史以来最伟大的基金经理彼得&#183;林奇凭借其在投资领域杰出的贡献，终其一生的经验和智慧，书写了这本《彼得&#183;林奇的成功投资》。这本书可以说是广大的业余投资者的福音，书中不仅讲述了业余投资者投资入门的基础知识，而且还深刻精辟的分析了业余投资者最常见的投资心理误区以及投资陷阱等，是一套最实用、最有效的股票投资方面的专业书籍，让人读后深受启发。",
    "作为一名山西太谷人，从小听多了有关晋商的故事，也去过许多山西的地方，但由于离家已久，故乡渐渐地模糊起来，不是山水，而是那一段段历史。想向人推荐或向人解说，却尴尬的发现，有许多东西已记不准确，近年来走的地方多，发现对于他乡的知识已超过故乡。于是诚惶诚怕的找来故乡的相关书籍进行系统的复习。这本书对各地的介绍虽然不够详细，但这种风格及语言却是我非常喜欢的，它使我重温了记忆中的故乡，增强了我对故乡历史的熟悉，更增加了一种作为客居异乡的山西人的自豪。我希望能跟着本书的脚步去重新游走故乡的山水。"
]
docs = []
# 对每个样本进行分词，并将分词结果添加到 docs 列表中
for review in reviews:
    docs.append(split_words(review))
# 将分词后的文档转换为词袋模型
corpus = [dct.doc2bow(line) for line in docs]
# 使用TF-IDF模型对转换后的语料库进行预测，计算每个词的TF-IDF值
rets = model[corpus]
# 打印每个文档的TF-IDF结果，包括长度和具体的TF-IDF值对
for item in rets:
    print (len(item), item)
# print (rets)
final_ret = transformedCorpus2Numpy(rets, len(reviews), len(dct))
print(final_ret.shape)
print(final_ret)

68 [(0, 0.020062233904168687), (11, 0.025639735126240674), (13, 0.03299742274045511), (24, 0.026866616210774394), (27, 0.021787915254789795), (55, 0.030103387941530328), (71, 0.05961353956518443), (84, 0.035628154166583555), (89, 0.02810256110535029), (95, 0.015201959638314368), (99, 0.03632849840060121), (104, 0.08528742624327215), (159, 0.027964698297023888), (202, 0.07344335797237768), (304, 0.1706508752494116), (325, 0.07056855178994123), (328, 0.06508877085055055), (334, 0.027230224042210576), (345, 0.04478784759315718), (351, 0.03466780888518984), (362, 0.05687715001638691), (363, 0.05680696693245626), (373, 0.08428435019859719), (385, 0.07541980146803437), (390, 0.08292372088953685), (416, 0.05750358530781197), (597, 0.06876352974153732), (614, 0.04365830957529404), (621, 0.01889827742517651), (721, 0.08428435019859719), (725, 0.10424503737510073), (732, 0.05454334610482109), (737, 0.06622557526236882), (851, 0.18175215830998415), (852, 0.1844219613878578), (853, 0.1326596302228

In [26]:
data_file_path = file_path
output_dir = os.path.join(base_path, "output/gensim/t2")
os.makedirs(output_dir, exist_ok=True)
# 1. 词典构建
docs = []
df = pd.read_csv(data_file_path)
n = 1000
for label in [0, 1]:
    _df = df[df.label == label][:n]
    documents = _df.review.values
    for document in tqdm(documents):
        # document = str(document)  # 类型强制转换为str
        try:
            docs.append(split_words(document))
        except Exception as e:
            print(document, "异常:", e)
dct = Dictionary(docs)
dct.save_as_text(os.path.join(output_dir , "dict.txt"))

# 2. 词袋法(TF)转换
corpus = [dct.doc2bow(line) for line in docs]

# 3. TFIDF模型构建
model = TfidfModel(corpus=corpus)
model.save(os.path.join(output_dir , "tfidf.pkl"))
corpus = model[corpus]  # TF-IDF转换/模型预测

# 4. LDA主题模型
# noinspection PyStatementEffect
dct[0]  # 需要触发id2token的构建
lda = LdaModel(corpus=corpus, id2word=dct.id2token.copy(), num_topics=2)
lda.save(os.path.join(output_dir , "lda.pkl"))

# 5. LDA的数据转换
features = lda[corpus]
print(type(features))

100%|██████████| 1000/1000 [00:00<00:00, 2692.14it/s]


<class 'gensim.interfaces.TransformedCorpus'>


In [11]:
data_file_path = file_path
output_dir = os.path.join(base_path, "output/gensim/t2")
os.makedirs(output_dir, exist_ok=True)
# 1. 词典构建
docs = []
df = pd.read_csv(data_file_path)
n = 5
for label in [0, 1]:
    _df = df[df.label == label][:n]
    documents = _df.review.values
    for document in tqdm(documents):
        # document = str(document)  # 类型强制转换为str
        try:
            docs.append(split_words(document))
        except Exception as e:
            print(document, "异常:", e)
print(docs)

# 1. 加载
dct: Dictionary = Dictionary.load_from_text(os.path.join(output_dir , "dict.txt"))
tfidf = TfidfModel.load(os.path.join(output_dir , "tfidf.pkl"))
lda: LdaModel = LdaModel.load(os.path.join(output_dir , "lda.pkl"))

# 2. 预测（文本特征向量）
corpus = [dct.doc2bow(line) for line in docs]  # 词袋法转换
corpus = tfidf[corpus]  # TF-IDF转换/模型预测
features = lda[corpus]  # LDA转换/预测
features = transformedCorpus2Numpy(features, n_docs=len(docs), n_features=lda.num_topics)
print(features)

# 3. 单词特征向量
# noinspection PyUnresolvedReferences
word_embedding_tabel = lda.get_topics().T
print(type(word_embedding_tabel))
print(word_embedding_tabel.shape)
print(word_embedding_tabel)

print(f"'很差'这个词对应的向量:{word_embedding_tabel[dct.token2id['很差']]}")

100%|██████████| 5/5 [00:00<00:00, 3036.27it/s]

[['做', '为', '一本', '声名在外', '的', '流行', '书', '，', '说', '的', '还是', '广州', '的', '外企', '，', '按', '道理', '应该', '和', '我', '的', '生存环境', '差不多', '啊', '。', '但是', '一看之下', '，', '才', '发现', '相去甚远', '。', '这', '也', '就算', '了', '，', '还', '发现', '其中', '的', '很多', '规则', '有', '很', '强', '的', '企业', '个性', '，', '也', '就', '说', '，', '只是', '个例', '，', '而', '不是', '行例', '。', '给', '我们', '这些', '老油条', '看看', '也', '就算', '了', '，', '如果', '给', '那些', '对', '外企', '向往', '，', '或者', '想', '了解', '的', 'freshman', '来看', '，', '实在', '是', '容易', '误导', '他们', '。'], ['作者', '有', '明显', '的', '自恋', '倾向', '，', '只有', '有', '老公', '养', '不', '上班', '的', '太太', '们', '才能', '像', '她', '那样', '生活', '。', '很多', '方法', '都', '不', '实用', '，', '还有', '抄袭', '嫌疑', '，', '没意思', '，', '没买', '的', '可以', '省省', '了', '。'], ['作者', '完全', '是', '以', '一个', '过来', '的', '自', '认为', '是', '成功者', '的', '角度', '去', '写', '这个', '问题', '，', '感觉', '很', '不', '客观', '。', '虽然', '不是', '很', '喜欢', '，', '但是', '，', '有', '不少', '的', '观点', '还是', '可取', '的', '。'], ['作者', '提倡', '内', '调', '，', '不', '信任', '化妆品', '，', '这